In [1]:
import numpy as np
import os
from matplotlib import pyplot as plt
import cv2
import random
import pickle

In [2]:
file_list, class_list = [], []

IMG_SIZE = 80
DATADIR = 'Train'
CATEGORIES = ["left","right","up","down"]

for category in CATEGORIES :
    path = os.path.join(DATADIR, category)
    for img in os.listdir(path):
        img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
        #plt.imshow(img_array)

training_data = []

def create_training_data():
    for category in CATEGORIES :
        path = os.path.join(DATADIR, category)
        class_num = CATEGORIES.index(category)
        for img in os.listdir(path):
            try :
                img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                #print(img_array,len(img_array))
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                training_data.append([new_array, class_num])
            except Exception as e:
                pass

create_training_data()
X, y = [], []

for features, label in training_data:
    X.append(features)
    y.append(label)

#print(len(X),len(y),X[0])

#print(y)


In [3]:
X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
#print(len(X[0]))
y = np.array(y)

# Creating the files containing all the information about your model
pickle_out = open("X.pickle", "wb")
pickle.dump(X, pickle_out)
pickle_out.close()

pickle_out = open("y.pickle", "wb")
pickle.dump(y, pickle_out)
pickle_out.close()

pickle_in = open("X.pickle", "rb")
X = pickle.load(pickle_in)

In [4]:
import tensorflow as tf 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
import pickle
from keras.models import model_from_json
from keras.models import load_model
import matplotlib.pyplot as plt

2022-10-03 21:51:54.934706: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-03 21:51:55.879028: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/sashankk/miniconda3/envs/vortexdetect/lib/python3.10/site-packages/cv2/../../lib64:
2022-10-03 21:51:55.879064: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-10-03 21:51:55.977800: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS w

In [5]:
# Opening the files about data
X = pickle.load(open("X.pickle", "rb"))
y = pickle.load(open("y.pickle", "rb"))

# normalizing data (a pixel goes from 0 to 255)
X = X/255.0

In [6]:
# Building the model
model = Sequential()
# 3 convolutional layers
model.add(Conv2D(32, (3, 3), input_shape = X.shape[1:]))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

# 2 hidden layers
model.add(Flatten())
model.add(Dense(128))
model.add(Activation("relu"))

model.add(Dense(128))
model.add(Activation("relu"))

# The output layer with 2 neurons, for 2 classes
model.add(Dense(4))
model.add(Activation("softmax"))

2022-10-03 21:51:59.580171: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/sashankk/miniconda3/envs/vortexdetect/lib/python3.10/site-packages/cv2/../../lib64:
2022-10-03 21:51:59.580763: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-10-03 21:51:59.580798: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (sashankk-VirtualBox): /proc/driver/nvidia/version does not exist
2022-10-03 21:51:59.581990: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 78, 78, 32)        320       
                                                                 
 activation (Activation)     (None, 78, 78, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 39, 39, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 37, 37, 64)        18496     
                                                                 
 activation_1 (Activation)   (None, 37, 37, 64)        0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 18, 18, 64)       0         
 2D)                                                    

In [8]:
# Compiling the model using some basic parameters
model.compile(loss="sparse_categorical_crossentropy",
                optimizer="adam",
                metrics=["accuracy"])

# Training the model, with 40 iterations
# validation_split corresponds to the percentage of images used for the validation phase compared to all the images
history = model.fit(X, y, batch_size=32, epochs=40, validation_split=0.05)

Epoch 1/40
14/14 [==============================] - 5s 282ms/step - loss: 1.3829 - accuracy: 0.2796 - val_loss: 1.7584 - val_accuracy: 0.0000e+00
Epoch 2/40
14/14 [==============================] - 4s 259ms/step - loss: 1.3473 - accuracy: 0.3445 - val_loss: 1.6634 - val_accuracy: 0.0000e+00
Epoch 3/40
14/14 [==============================] - 4s 254ms/step - loss: 1.1324 - accuracy: 0.5280 - val_loss: 1.0244 - val_accuracy: 0.3333
Epoch 4/40
14/14 [==============================] - 4s 255ms/step - loss: 0.9201 - accuracy: 0.5347 - val_loss: 0.8167 - val_accuracy: 0.6250
Epoch 5/40
14/14 [==============================] - 4s 251ms/step - loss: 0.8335 - accuracy: 0.5861 - val_loss: 0.5856 - val_accuracy: 0.7917
Epoch 6/40
14/14 [==============================] - 4s 253ms/step - loss: 0.7473 - accuracy: 0.5794 - val_loss: 0.9627 - val_accuracy: 0.1667
Epoch 7/40
14/14 [==============================] - 4s 253ms/step - loss: 0.7293 - accuracy: 0.5884 - val_loss: 1.0839 - val_accuracy: 0.125

In [9]:
model_json = model.to_json()
with open("model.json", "w") as json_file :
    json_file.write(model_json)

model.save_weights("model.h5")
print("Saved model to disk")

model.save('CNN.model')

Saved model to disk


FailedPreconditionError: CNN.model is not a directory

In [ ]:
print(history.history.keys())
plt.figure(1)
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')